In [1]:
import warnings
warnings.filterwarnings("ignore")

from octopus.db import PostgresqlManager
from dotenv import load_dotenv

import matplotlib.pyplot as plt
from tqdm import tqdm

# Roadmap

1. Prepare links first
   - transactions (bio-ticker)
   - committee-assignment (bio-committee)
   - lobbying-on-bills (ticker-bill)
   - bill-assignments (bill-committee)
   - sponsors (bio-bill)
      - Since we have 115, 116, 117th committee assignments, let's cut-off from 2016-01-01 []

In [2]:
# get transactions data (bioguide-ticker)
load_dotenv("/Users/syyun/Dropbox (MIT)/efd/.envlv", override=True)
pm = PostgresqlManager(dotenv_path="/Users/syyun/Dropbox (MIT)/efd/.envlv")
trans = pm.execute_sql(fetchall=True, sql=
                    """
                    with st as (
                    select distinct bioguide_id, ticker, trans_date  from "_sandbox_suyeol".senate_annual_4a saa
                        inner join  _sandbox_suyeol.senate_annual sa on sa.report_type_url = saa.report_url
                        inner join _sandbox_suyeol.senator_bioguide sb on sb.first_name = sa.first_name  and sb.last_name = sa.last_name 
                    where ticker is not null and trans_date is not null
                    )
                    , ht as (
                    select distinct bioguide_id, ticker, transaction_date as trans_date from "_sandbox_suyeol".house_fd_transactions hft 
                    	inner join "_sandbox_suyeol".house_docs_id hdi on hdi."DocID" =hft.docid 
                    	inner join "_sandbox_suyeol".house_fd_bio_brdige hfbb on hfbb.first_name = hdi."First" and hfbb.last_name =hdi."Last" 
					where ticker is not null and transaction_date is not null
					)
					, union_sh as (
					select * from st
					union
					select * from ht
					)
					select * from union_sh        
                    """)

In [3]:
import pandas as pd

# Create a DataFrame with the desired column names
trans_df = pd.DataFrame(trans, columns=['bioguide_id', 'ticker', 'transaction_date'])

# Convert transaction date to datetime
trans_df['transaction_date'] = pd.to_datetime(trans_df['transaction_date'])

# Cleanse ticker by splitting on whitespace, stripping, and taking the first part
trans_df['ticker'] = trans_df['ticker'].apply(lambda x: x.strip().split()[0].strip())

# Display the cleaned DataFrame
print(trans_df.shape)
trans_df.head(10)

(24675, 3)


,bioguide_id,ticker,transaction_date
0,W000802,VOD,2016-09-28
1,Y000062,YUM,2018-03-08
2,Y000062,MCD,2017-11-09
3,P000608,SYK,2015-06-23
4,C001101,UBS,2021-09-16
5,R000609,LLY,2019-02-19
6,F000462,TFC,2020-06-10
7,W000779,MAR,2020-06-05
8,C001066,DIS,2016-06-06
9,G000583,CHUY,2020-09-15


In [4]:
# get committee assignments
cgnum = 109 # this covers 2014
assign = pm.execute_sql(fetchall=True, sql=
                f"""
                select distinct committee_thomas_id, legislator_bioguide_id, congress_num  from relational___congress.committees__legislators cl
	                inner join relational___congress.legislators l on l.bioguide_id = cl.legislator_bioguide_id                """
                )

In [5]:
import pandas as pd

# Create a DataFrame with the desired column names
assign_df = pd.DataFrame(assign, columns=['committee_id', 'bioguide_id', 'congress_num'])

# Display the cleaned DataFrame
print(assign_df.shape)
assign_df.head(10)

(11698, 3)


,committee_id,bioguide_id,congress_num
0,SSBK08,K000393,115
1,HSBA13,P000616,116
2,SSBK12,B000944,115
3,HSAP04,R000609,115
4,SSFR,C000141,116
5,HSPW14,B001295,117
6,HSGO24,C001116,117
7,HSED02,S001206,116
8,SSEG07,L000577,117
9,HSPW,L000578,117


In [6]:
import pandas as pd

# Load your dataframe, assuming it is called df
assign_df['congress_num'] = pd.to_numeric(assign_df['congress_num']) # convert congress_num to numeric

# Define function to get start and end year of congress
def get_congress_years(congress_num):
    start_year = (congress_num - 115) * 2 + 2017
    end_year = start_year + 1
    return start_year, end_year

# Apply function to congress_num column to get year start and end dates
assign_df = assign_df.assign(congress_years=assign_df['congress_num'].apply(get_congress_years))

# Split the congress_years column into two separate columns
assign_df[['congress_year_start', 'congress_year_end']] = pd.DataFrame(assign_df['congress_years'].tolist(), index=assign_df.index)

assign_df['congress_year_start'] = pd.to_datetime(assign_df['congress_year_start'].astype(str) + '-01-03')
assign_df['congress_year_end'] = pd.to_datetime((assign_df['congress_year_end']+1).astype(str) + '-01-03')


# Drop the congress_years column
assign_df.drop('congress_years', axis=1, inplace=True)

assign_df.head(10)


,committee_id,bioguide_id,congress_num,congress_year_start,congress_year_end
0,SSBK08,K000393,115,2017-01-03,2019-01-03
1,HSBA13,P000616,116,2019-01-03,2021-01-03
2,SSBK12,B000944,115,2017-01-03,2019-01-03
3,HSAP04,R000609,115,2017-01-03,2019-01-03
4,SSFR,C000141,116,2019-01-03,2021-01-03
5,HSPW14,B001295,117,2021-01-03,2023-01-03
6,HSGO24,C001116,117,2021-01-03,2023-01-03
7,HSED02,S001206,116,2019-01-03,2021-01-03
8,SSEG07,L000577,117,2021-01-03,2023-01-03
9,HSPW,L000578,117,2021-01-03,2023-01-03


In [7]:
# ticker's lobbying on bills
load_dotenv("/Users/syyun/Dropbox (MIT)/efd/.envlv", override=True)
pm = PostgresqlManager(dotenv_path="/Users/syyun/Dropbox (MIT)/efd/.envlv")
lob = pm.execute_sql(fetchall=True, sql=
                f"""
                with lobby_on_bill as (
                    select distinct concat(bill_type, bill_number_chain, '-', congress_num) as bill_id, ticker, f.dt_posted  from link___lda__congress."_issue_paragraphs__bills" ipb 
                    inner join relational___lda.clientships c on c.clientship_id  = ipb.clientship_id 
                    inner join relational___lda.filings f on f.filing_uuid  = ipb.filing_uuid 
                    inner join "_sandbox_suyeol".client_ticker ct on ct.client_name = c.client_name 
                    where ticker != 'Not found' and ticker is not null
                    )
                    select * from lobby_on_bill
                """
                )

In [8]:
# Create the Pandas dataframe
lob_df = pd.DataFrame(lob, columns=['bill_id', 'ticker', 'datetime'])

# Convert datetime column to date only
lob_df['date'] = pd.to_datetime(lob_df['datetime']).dt.date

# Drop the datetime column
lob_df.drop('datetime', axis=1, inplace=True)

# Print the resulting dataframe
lob_df.head(10)

,bill_id,ticker,date
0,hr1332-116,HCA,2019-10-18
1,hr3962-111,IVCRQ,2010-04-16
2,hr1-115,INVVY,2018-01-18
3,hr3590-111,SVNDY,2010-04-20
4,hr842-117,HLF,2021-10-19
5,hr2113-116,CSL,2019-10-17
6,s684-116,HMRK,2019-10-18
7,hr5497-112,PG,2012-10-19
8,hr767-108,RTX,2005-02-11
9,hr2471-111,PNW,2010-01-20


In [9]:
# bill-assignments (bill-committee)
ba = pm.execute_sql(fetchall=True, sql=
                f"""
select b.bill_id, committee_id, b.introduced_at  from relational___congress.bills__committees bc
      	inner join relational___congress.bills b on b.bill_id = bc.bill_id 
                """)

In [10]:
# Create the Pandas dataframe
ba_df = pd.DataFrame(ba, columns=['bill_id', 'committee_id', 'intro_date'])

# Convert datetime column to date only
ba_df['date'] = pd.to_datetime(ba_df['intro_date']).dt.date

# Drop the datetime column
ba_df.drop('intro_date', axis=1, inplace=True)

# Print the resulting dataframe
print(ba_df.shape)
ba_df.head(10)

(546703, 3)


,bill_id,committee_id,date
0,hres607-93,HSFA,1973-10-17
1,hres556-93,HSFA,1973-09-20
2,hres322-93,HSRU,1973-03-22
3,hres18-93,HSRU,1973-01-03
4,hres482-93,HSFA,1973-07-10
5,hres350-93,HSWM,1973-04-10
6,hres528-93,HSAS,1973-08-03
7,hres1408-93,HSFA,1974-10-02
8,hres226-93,HSRU,1973-02-20
9,hres1244-93,HSRU,1974-07-18


In [18]:
# bill-sponsor
bs = pm.execute_sql(fetchall=True, sql=
                f"""
      	select b.bill_id, legislator_bioguide_id, introduced_at from relational___congress.bills__legislators bl
      		inner join relational___congress.bills b on bl.bill_id = b.bill_id 
      	where relation = 'sponsor'
                """)

In [20]:
# Create the Pandas dataframe
bs_df = pd.DataFrame(bs, columns=['bill_id', 'bioguide_id', 'intro_date'])

# Convert datetime column to date only
bs_df['date'] = pd.to_datetime(bs_df['intro_date']).dt.date

# Drop the datetime column
bs_df.drop('intro_date', axis=1, inplace=True)

# Print the resulting dataframe
print(bs_df.shape)
bs_df.head(10)

(335902, 3)


,bill_id,bioguide_id,date
0,hconres100-105,D000275,1997-06-18
1,hconres100-100,R000053,1987-04-08
2,hconres100-101,Y000014,1989-04-18
3,hconres100-102,M000590,1991-03-19
4,hconres100-103,B000403,1993-05-12
5,hconres100-106,B000463,1999-05-06
6,hconres100-104,H000981,1995-09-06
7,hconres100-111,B001230,2009-04-21
8,hconres100-107,D000533,2001-04-04
9,hconres100-108,M001147,2003-03-18


In [21]:
# bill-cosponsor
bcs = pm.execute_sql(fetchall=True, sql=
                f"""
      	select b.bill_id, legislator_bioguide_id, introduced_at from relational___congress.bills__legislators bl
      		inner join relational___congress.bills b on bl.bill_id = b.bill_id 
      	where relation = 'cosponsor'
                """)

In [ ]:
# Create the Pandas dataframe
bcs_df = pd.DataFrame(bcs, columns=['bill_id', 'bioguide_id', 'intro_date'])

# Convert datetime column to date only
bcs_df['date'] = pd.to_datetime(bcs_df['intro_date']).dt.date

# Drop the datetime column
bcs_df.drop('intro_date', axis=1, inplace=True)

# Print the resulting dataframe
print(bcs_df.shape)
bcs_df.head(10)

(3635006, 2)


,bill_id,bioguide_id
0,hconres100-100,G000445
1,hconres100-101,B000403
2,hconres100-101,M000015
3,hconres100-102,A000014
4,hconres100-102,A000022
5,hconres100-102,A000189
6,hconres100-102,B000229
7,hconres100-102,B000463
8,hconres100-102,B000551
9,hconres100-102,B000586


In [13]:
import networkx as nx
import torch
from torch_geometric.utils import from_networkx

# Create an empty graph
G = nx.Graph()

# Define a reference date
reference_date = pd.Timestamp('2000-01-01')

# Assign unique integer IDs to each bioguide_id
bioguide_id_mapping = {bioguide_id: i for i, bioguide_id in enumerate(trans_df['bioguide_id'].unique())}
print(bioguide_id_mapping)

# Iterate through the DataFrame rows, adding nodes and edges
for index, row in trans_df.iterrows():
    bioguide_id = row['bioguide_id']
    ticker = row['ticker']
    transaction_date = row['transaction_date']
    
    # Calculate the number of days elapsed since the reference date
    days_elapsed = (transaction_date - reference_date).days

    # Add nodes with features
    G.add_node(bioguide_id, node_type='person', unique_id=bioguide_id_mapping[bioguide_id])
    G.add_node(ticker, node_type='ticker')

    # Add edges with features (e.g., [days_elapsed, days_elapsed])
    G.add_edge(bioguide_id, ticker, date=[days_elapsed, days_elapsed])

# Map categorical node and edge features to numerical values
node_types = {node: data['node_type'] for node, data in G.nodes(data=True)}
unique_node_types = set(node_types.values())
node_type_mapping = {node_type: i for i, node_type in enumerate(unique_node_types)}
numerical_node_types = {node: node_type_mapping[node_type] for node, node_type in node_types.items()}

# Assign numerical node features to the graph
nx.set_node_attributes(G, numerical_node_types, name='node_type')

# Convert the nx.Graph object to a torch_geometric.data.Data object
data = from_networkx(G)
data.x = torch.tensor([data.node_type[i] for i in range(data.num_nodes)], dtype=torch.long).view(-1, 1)

{'W000802': 0, 'Y000062': 1, 'P000608': 2, 'C001101': 3, 'R000609': 4, 'F000462': 5, 'W000779': 6, 'C001066': 7, 'G000583': 8, 'P000612': 9, 'L000559': 10, 'M000934': 11, 'P000616': 12, 'L000579': 13, 'H000636': 14, 'B001255': 15, 'B001292': 16, 'R000307': 17, 'T000488': 18, 'N000192': 19, 'C001123': 20, 'M001180': 21, 'C001069': 22, 'L000397': 23, 'L000590': 24, 'C001120': 25, 'G000545': 26, 'M001111': 27, 'F000450': 28, 'W000804': 29, 'G000584': 30, 'C001047': 31, 'B000574': 32, 'B001274': 33, 'J000305': 34, 'A000076': 35, 'S001201': 36, 'S000583': 37, 'H000615': 38, 'K000383': 39, 'P000197': 40, 'J000020': 41, 'C001114': 42, 'M001199': 43, 'F000461': 44, 'M001205': 45, 'T000461': 46, 'G000596': 47, 'M001135': 48, 'C001035': 49, 'F000468': 50, 'S000250': 51, 'M001193': 52, 'M001213': 53, 'D000624': 54, 'G000599': 55, 'C001062': 56, 'D000399': 57, 'S001156': 58, 'C001107': 59, 'M001198': 60, 'H001082': 61, 'G000597': 62, 'H001061': 63, 'A000364': 64, 'P000523': 65, 'M001203': 66, 'S00

ValueError: Not all nodes contain the same attributes